# [실습] LangChain을 이용한 데이터 생성과 처리


LCEL의 기본 문법인 Prompt | llm | Parser 구조에 대해 배웠습니다.   
이번 실습에서는 출력을 구조화하고, LLM을 연결하는 방법에 대해 알아봅니다.


### 라이브러리 설치  

랭체인 OpenAI와 Google 모듈을 설치합니다.

In [ ]:
!pip install langchain langchain_community google-generativeai langchain_google_genai langchain_openai openai dotenv arxiv pymupdf

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv('env', override=True)

if os.environ.get('OPENAI_API_KEY'):
    print('OpenAI API 키 확인')
if os.environ.get('GOOGLE_API_KEY'):
    print('Google API 키 확인')

OpenAI API 키 확인
Google API 키 확인


### LLM 모델 불러오기

In [ ]:
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain.chat_models import init_chat_model

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.167,  # 분당 10개 요청
    check_every_n_seconds=0.1,  # 100ms마다 체크
    max_bucket_size=10,  # 최대 버스트 크기
)

gpt_llm = init_chat_model(
    "gpt-5-mini", model_provider="openai", reasoning_effort = 'low', temperature=0)

gemini_llm = init_chat_model(
    "gemini-2.5-flash", model_provider="google_genai", temperature=0, thinking_budget=1000, rate_limiter=rate_limiter
)
gpt_llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7d40af9147a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7d40aea64e00>, root_client=<openai.OpenAI object at 0x7d40af09a060>, root_async_client=<openai.AsyncOpenAI object at 0x7d40aecbed20>, model_name='gpt-5-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True, reasoning_effort='low')

## JsonOutputParser 로 Json 형식의 출력 만들기

LLM의 출력을 구조화하면, 데이터 후처리를 하지 않고도 다른 코드와 연결할 수 있습니다.   
JSON 형식의 출력을 구성해 보겠습니다.

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

jsonparser = JsonOutputParser()

JSON 파서의 역할은 JSON 규격에 맞는 텍스트를 Dict 형식으로 변환하는 것으로,     
실제 형식에 대한 조건을 프롬프트로 전달해야 합니다.

In [ ]:
jsonparser.get_format_instructions()

'Return a JSON object.'

In [ ]:
recipe_template = ChatPromptTemplate([
    ('system','당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'),
    ('user','''저는 다음의 재료와 조건을 이용한 환상적인 퓨전 다이닝을 만들고 싶습니다. 1가지 메뉴만 추천해주세요!
레시피에 대한 정보를 JSON 형식으로 출력해주세요.

[재료]: {ingredient}
[조건]: {condition}
''')
])

recipe_chain = recipe_template | gemini_llm | jsonparser

In [ ]:
response = recipe_chain.invoke({'ingredient':'콜라, 고수', 'condition':'디저트'})
response

{'dish_name': '콜라 고수 판나코타와 라임 실란트로 시럽',
 'description': '부드러운 콜라 판나코타에 신선한 고수와 라임의 향긋함이 더해진 시럽을 곁들인 디저트입니다. 콜라의 캐러멜 향과 고수의 상큼하고 독특한 풍미가 만나 이색적이면서도 조화로운 맛의 균형을 이룹니다. 예상치 못한 조합이 선사하는 미식의 즐거움을 경험해보세요.',
 'ingredients': {'콜라 판나코타': [{'item': '콜라', 'quantity': '500ml'},
   {'item': '생크림', 'quantity': '200ml'},
   {'item': '설탕', 'quantity': '50g (콜라의 당도에 따라 조절)'},
   {'item': '판젤라틴', 'quantity': '5g (또는 가루젤라틴 5g)'}],
  '라임 실란트로 시럽': [{'item': '고수', 'quantity': '20g (줄기 포함)'},
   {'item': '라임 즙', 'quantity': '30ml'},
   {'item': '설탕', 'quantity': '30g'},
   {'item': '물', 'quantity': '50ml'}],
  '가니쉬': [{'item': '신선한 고수 잎', 'quantity': '약간'},
   {'item': '라임 제스트', 'quantity': '약간'}]},
 'instructions': [{'step': 1,
   'description': '콜라 판나코타 준비: 콜라 500ml를 냄비에 넣고 중불에서 끓여 약 1/3 정도로 졸여줍니다 (약 150ml). 콜라의 향이 응축되고 단맛이 강해집니다. 식혀둡니다.'},
  {'step': 2,
   'description': '판젤라틴은 찬물에 5분간 불려 부드럽게 만듭니다. 가루젤라틴을 사용할 경우, 찬물 30ml에 젤라틴을 뿌려 5분간 불려둡니다.'},
  {'step': 3,
   'description': '다른 냄비에 생크림과 설탕 50g을 넣고 약불에서 

In [ ]:
# Dict 구조: 추출 가능
response['ingredients']

{'콜라 판나코타': [{'item': '콜라', 'quantity': '500ml'},
  {'item': '생크림', 'quantity': '200ml'},
  {'item': '설탕', 'quantity': '50g (콜라의 당도에 따라 조절)'},
  {'item': '판젤라틴', 'quantity': '5g (또는 가루젤라틴 5g)'}],
 '라임 실란트로 시럽': [{'item': '고수', 'quantity': '20g (줄기 포함)'},
  {'item': '라임 즙', 'quantity': '30ml'},
  {'item': '설탕', 'quantity': '30g'},
  {'item': '물', 'quantity': '50ml'}],
 '가니쉬': [{'item': '신선한 고수 잎', 'quantity': '약간'},
  {'item': '라임 제스트', 'quantity': '약간'}]}

Json으로 파싱하는 방법은 활용도가 높지만, 실행할 때마다 결과뿐만 아니라 형식도 달라진다는 문제가 있습니다.

In [ ]:
response = recipe_chain.invoke({'ingredient':'문어, 피넛버터', 'condition':'메인 요리'})
response

{'dishName': '문어 땅콩 사테 구이와 고추장 코코넛 폴렌타',
 'description': '지중해의 신선한 문어와 동남아시아의 고소한 땅콩 사테 소스, 그리고 한국의 매콤한 고추장이 만나 탄생한 환상적인 퓨전 메인 요리입니다. 부드럽게 익힌 문어를 직화로 구워 불맛을 입히고, 코코넛 밀크와 고추장으로 풍미를 더한 크리미한 폴렌타 위에 올려냅니다. 고소하고 매콤달콤한 땅콩 사테 소스가 문어의 감칠맛을 극대화하며, 이색적이면서도 익숙한 맛의 조화를 선사합니다.',
 'cuisineType': '퓨전 (한식/동남아/지중해)',
 'mainIngredients': ['문어', '피넛버터', '코코넛 밀크', '고추장', '폴렌타'],
 'prepTime': '30분',
 'cookTime': '1시간 30분 (문어 삶는 시간 포함)',
 'servings': '2인분',
 'difficulty': '중급',
 'ingredients': {'문어_구이': [{'item': '자숙 문어 (또는 생문어)', 'quantity': '400g'},
   {'item': '올리브 오일', 'quantity': '2큰술'},
   {'item': '파프리카 가루', 'quantity': '1작은술'},
   {'item': '소금', 'quantity': '약간'},
   {'item': '후추', 'quantity': '약간'}],
  '땅콩_고추장_사테_소스': [{'item': '피넛버터 (무가당)', 'quantity': '3큰술'},
   {'item': '코코넛 밀크', 'quantity': '50ml'},
   {'item': '고추장', 'quantity': '1큰술'},
   {'item': '간장', 'quantity': '1큰술'},
   {'item': '라임 즙', 'quantity': '1큰술'},
   {'item': '다진 마늘', 'quantity': '1작은술'},
   {'item': '다진 생강', 'quantity': '1/2작은

## Pydantic을 이용해 출력 형식 지정하기

pydantic은 데이터 형식에 제약조건을 두고 이를 준수하는지 검증하는 라이브러리입니다.


In [ ]:
from pydantic import BaseModel, Field
# pydantic 연동

class Recipe(BaseModel):
    name: str = Field(description="음식 이름")
    # name: 문자열, 설명은 "음식 이름"
    difficulty: str = Field(description="만들기의 난이도")

    origin: str = Field(description="원산지")
    ingredients: list[str] = Field(description="재료")
    # ingredients: 문자열 리스트, 설명은 "재료"

    instructions: list[str] = Field(description="조리법")
    tip: str = Field(description='조리 과정 팁')


In [ ]:
parser = JsonOutputParser(pydantic_object=Recipe)

In [ ]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "음식 이름", "title": "Name", "type": "string"}, "difficulty": {"description": "만들기의 난이도", "title": "Difficulty", "type": "string"}, "origin": {"description": "원산지", "title": "Origin", "type": "string"}, "ingredients": {"description": "재료", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "instructions": {"description": "조리법", "items": {"type": "string"}, "title": "Instructions", "type": "array"}, "tip": {"description": "조리 과정 팁", "title": "Tip", "type": "string"}}, "required": ["name",

해당 내용을 프롬프트에 포함합니다.

In [ ]:
recipe_template2 = ChatPromptTemplate([
    ('system','당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'),
    ('user','''저는 다음의 재료를 이용한 실험적인 음식을 만들고 싶습니다. 추천해주세요!
    레시피에 대한 정보를 JSON 형식으로 출력해주세요. 결과는 한국어로 작성하세요.
재료: {ingredient}
출력 형식 조건: {instruction}''')
])

recipe_chain2 = recipe_template2 | gemini_llm | parser


In [ ]:
recipe_chain2.invoke({'ingredient':'생강', 'instruction':parser.get_format_instructions()})

{'name': '생강 코코넛 판나코타와 망고 살사',
 'difficulty': '중간',
 'origin': '이탈리아 & 동남아시아 퓨전',
 'ingredients': ['코코넛 밀크 400ml',
  '생크림 200ml',
  '설탕 50g (기호에 따라 조절)',
  '판 젤라틴 3장 (또는 가루 젤라틴 5g)',
  '신선한 생강 30g (껍질 벗겨 얇게 저미거나 즙을 낼 것)',
  '잘 익은 망고 1개',
  '라임 즙 1큰술',
  '고수 1큰술 (다진 것, 선택 사항)',
  '작은 붉은 양파 1/4개 (아주 잘게 다진 것, 선택 사항)',
  '붉은 고추 약간 (씨 제거 후 다진 것, 선택 사항)',
  '민트 잎 약간 (장식용)'],
 'instructions': ['**생강 코코넛 판나코타 준비:**',
  '판 젤라틴은 찬물에 5-10분간 불려 부드럽게 만듭니다. 가루 젤라틴을 사용할 경우, 찬물 2-3큰술에 뿌려 5분간 불립니다.',
  '냄비에 코코넛 밀크, 생크림, 설탕을 넣고 중약불에서 설탕이 녹을 때까지 저어줍니다. 끓이지 않도록 주의하세요.',
  '신선한 생강은 껍질을 벗겨 얇게 저미거나 강판에 갈아 면포에 짜서 즙을 냅니다. (얇게 저민 생강을 사용할 경우, 이 단계에서 코코넛 밀크 혼합물에 넣고 함께 우려냅니다.)',
  '따뜻해진 코코넛 밀크 혼합물에 생강 즙(또는 우려낸 생강)을 넣고 5분 정도 더 우려냅니다.',
  '불린 젤라틴의 물기를 꼭 짜서 따뜻한 코코넛 밀크 혼합물에 넣고 완전히 녹을 때까지 잘 저어줍니다.',
  '생강 조각을 사용했다면 건져내고, 혼합물을 고운 체에 걸러 부드러운 질감을 만듭니다.',
  '준비된 컵이나 틀에 판나코타 혼합물을 붓고, 냉장고에서 최소 4시간 또는 밤새 굳힙니다.',
  '**망고 살사 준비:**',
  '망고는 껍질을 벗기고 씨를 제거한 후 작은 주사위 모양으로 썰어줍니다.',
  '작은 볼에 썬 망고, 라임 즙을 넣고 잘 섞습니다.',
  '기호에 따라 다진 고수, 

partial을 통해 먼저 일부를 입력할 수도 있습니다.

In [ ]:
recipe_template2 = ChatPromptTemplate([
    ('system','당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'),
    ('user','''저는 {ingredient}를 이용한 실험적인 음식을 만들고 싶습니다. 추천해주세요!
    레시피에 대한 정보를 JSON 형식으로 출력해주세요. 결과는 한국어로 작성하세요.
     {instruction}''')
]).partial(instruction=parser.get_format_instructions())

recipe_chain2 = recipe_template2 | gpt_llm | parser

recipe_chain2.invoke('감자')

{'name': '김치-카르파초 포테이토 롤',
 'difficulty': '중간',
 'origin': '한국 × 이탈리아 퓨전',
 'ingredients': ['중간 크기 감자 4개(껍질 제거)',
  '익힌 김치 150g(물기 걷어내고 잘게 썬 것)',
  '리코타 치즈 100g',
  '엑스트라버진 올리브오일 2큰술',
  '레몬즙 1큰술',
  '간장 1작은술',
  '참기름 1작은술',
  '다진 쪽파 2큰술',
  '통깨 1작은술',
  '소금·후추 약간',
  '올리브오일(팬용) 1큰술',
  '루꼴라 또는 어린잎(선택)',
  '얇게 썬 훈제 연어 또는 얇게 구운 슬라이스 비프(선택, 채식 옵션: 얇게 구운 가지)',
  '레몬 제스트 약간(선택)'],
 'instructions': ['감자를 얇게 슬라이스한다(두께 1~1.5mm 권장). 만돌린이 있으면 사용하면 균일하게 나온다.',
  '슬라이스한 감자를 소금물에 2분간 데쳐 녹말을 일부 제거한 뒤 찬물에 헹구고 키친타월로 물기를 잘 닦는다.',
  '오븐을 180°C로 예열한다. 베이킹 시트에 유산지를 깔고 감자 슬라이스를 겹치지 않게 펼쳐 올리브오일 살짝 바른 뒤 8~10분 가볍게 구워 말랑해지게 한다(너무 바삭해지면 말기 어려움).',
  '그동안 속재료를 준비한다: 잘게 썬 김치를 체에 밭쳐 물기와 과도한 유산을 제거한 뒤 리코타 치즈와 섞는다. 여기에 레몬즙 1작은술, 간장 1작은술, 참기름 약간, 다진 쪽파, 소금·후추로 간을 맞춘다.',
  '오븐에서 감자 슬라이스를 꺼내 살짝 식힌다. 손으로 다룰 수 있을 정도의 온도여야 한다.',
  '감자 한 장 위에 속재료를 1~2작은술 놓고 얇게 펼친 뒤(끝 부분은 적게), 선택한 훈제 연어나 슬라이스 비프/구운 가지를 얹는다. 감자를 말듯이 단단히 롤 말아 작은 타워 모양이나 롤 형태로 만든다.',
  '팬에 올리브오일을 두르고 약한 불에서 감자 롤 외피가 살짝 노릇해지도록 굴려 가며 1~2분간 익힌다(겉면에 색과 바삭함을 더함).'

# LangChain Structured Output
파서를 사용하지 않고, 구조화된 출력을 생성합니다.  

In [ ]:
from rich import print as rprint
structured_llm = gpt_llm.with_structured_output(Recipe)

rprint(structured_llm)

RunnableSequence(
    first=RunnableBinding(
        bound=ChatOpenAI(
            client=<openai.resources.chat.completions.completions.Completions object at 0x7d40af9147a0>,
            async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7d40aea64e00>,
            root_client=<openai.OpenAI object at 0x7d40af09a060>,
            root_async_client=<openai.AsyncOpenAI object at 0x7d40aecbed20>,
            model_name='gpt-5-mini',
            model_kwargs={},
            openai_api_key=SecretStr('**********'),
            stream_usage=True,
            reasoning_effort='low'
        ),
        kwargs={
            'response_format': <class '__main__.Recipe'>,
            'ls_structured_output_format': {
                'kwargs': {'method': 'json_schema', 'strict': None},
                'schema': {
                    'type': 'function',
                    'function': {
                        'name': 'Recipe',
                        'description': '',
                        'parameters': {
                            'properties': {
                                'name': {'description': '음식 이름', 'type': 'string'},
                                'difficulty': {'description': '만들기의 난이도', 'type': 'string'},
                                'origin': {'description': '원산지', 'type': 'string'},
                                'ingredients': {
                                    'description': '재료',
                                    'items': {'type': 'string'},
                                    'type': 'array'
                                },
                                'instructions': {
                                    'description': '조리법',
                                    'items': {'type': 'string'},
                                    'type': 'array'
                                },
                                'tip': {'description': '조리 과정 팁', 'type': 'string'}
                            },
                            'required': ['name', 'difficulty', 'origin', 'ingredients', 'instructions', 'tip'],
                            'type': 'object'
                        }
                    }
                }
            }
        },
        config={},
        config_factories=[]
    ),
    middle=[],
    last=RunnableBinding(
        bound=RunnableLambda(...),
        kwargs={},
        config={},
        config_factories=[],
        custom_output_type=<class '__main__.Recipe'>
    )
)

In [ ]:
response = structured_llm.invoke("생강으로 만들 수 있는 요리 레시피 알려주세요.")
rprint(response)

Recipe(
    name='생강차 (진저 티)',
    difficulty='쉬움',
    origin='한국',
    ingredients=['생강 30g', '물 500ml', '꿀 또는 설탕 1-2 큰술(기호에 따라)', '레몬 슬라이스(선택)'],
    instructions=[
        '생강은 껍질을 솔로 깨끗이 문지르거나 필요하면 얇게 껍질을 벗긴다.',
        '생강을 얇게 편으로 썰거나 강판에 곱게 갈아 준비한다.',
        '냄비에 물 500ml를 붓고 준비한 생강을 넣는다.',
        '중불에서 끓이다가 끓기 시작하면 불을 약하게 줄이고 10~15분간 은근히 더 끓여 생강 향을 우려낸다.',
        '불을 끄고 체로 걸러 생강 조각을 제거한다.',
        '기호에 따라 꿀 또는 설탕을 넣어 단맛을 맞추고, 레몬 슬라이스를 추가하면 상큼한 풍미가 난다.',
        '따뜻하게 혹은 얼음과 함께 시원하게 즐긴다.'
    ],
    tip='생강을 오래 우릴수록 매운맛은 줄고 은은한 단맛과 향이 강해진다. 더 진한 맛을 원하면 생강 양을 늘리거나 
우림 시간을 20분까지 늘려보세요. 꿀은 너무 뜨거울 때 넣으면 효능이 떨어질 수 있으니 불을 끈 뒤 약간 식힌 뒤 넣는 
것이 좋습니다.'
)

해당 출력은 Pydantic 클래스 형식으로 생성됩니다.   
with_structured_output 기능은 HuggingFace 계열의 오픈 모델에서는 지원되지 않으므로,    
이 때는 PydanticOutputParser를 사용해야 합니다.

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object = Recipe)

recipe_template =ChatPromptTemplate([
    ('system','당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'),
    ('user','''저는 {ingredient}를 이용한 실험적인 음식을 만들고 싶습니다. 추천해주세요!
    레시피에 대한 정보를 JSON 형식으로 출력해주세요. 결과는 한국어로 작성하세요.
     {instruction}''')
])

structured_llm2 = recipe_template.partial(instruction = parser.get_format_instructions()) | gpt_llm | parser

response = structured_llm2.invoke('커피')
response

Recipe(name='에스프레소 흑임자 파르페 (Espresso + Black Sesame Parfait)', difficulty='중간', origin='한국 × 프랑스 퓨전', ingredients=['에스프레소 샷 60ml (진한 에스프레소 2샷)', '우유 150ml (또는 바닐라 오트밀크)', '생크림 150ml', '연유 40g', '플레인 요거트 80g', '흑임자 페이스트 50g', '크런치 (미숫가루 또는 설탕에 구운 아몬드) 60g', '바닐라 시럽 10g (선택)', '카카오 닙스 또는 다크초콜릿 칩 20g', '젤라틴 파우더 2g (또는 아가아가 1g, 선택)', '물 10ml (젤라틴용)', '소금 한 꼬집', '코코아 파우더 약간 (장식용)', '신선한 민트잎 1-2장 (장식)'], instructions=['에스프레소를 추출해 식혀둔다. 에스프레소가 너무 뜨거우면 크림이 분리될 수 있으니 완전히 식혀야 한다.', '젤라틴을 사용할 경우, 차가운 물 10ml에 젤라틴 파우더를 뿌려 5분 불린 뒤 중탕 또는 전자레인지(10초 간격)로 녹여 준비한다. 아가아가 사용 시 포장지 지침에 따라 불리고 녹인다.', '생크림과 연유, 소금 한 꼬집을 볼에 넣고 부드러운 단단함이 남을 정도로 6~7부 거품으로 휘핑한다(너무 단단히 휘핑하지 않음).', '그릇에 플레인 요거트와 흑임자 페이스트를 섞어 크리미한 흑임자 베이스를 만든다. 필요하면 바닐라 시럽을 소량 넣어 단맛을 조절한다.', '휘핑한 크림의 1/3을 흑임자 베이스에 넣고 부드럽게 섞어 질감을 맞춘다. (이렇게 하면 베이스가 묽어지지 않음)', '식힌 에스프레소에 녹인 젤라틴을 빠르게 섞어 에스프레소 젤 레이어를 만든다. 젤라틴을 사용하지 않으면 이 단계는 생략 가능하나 레이어가 흐를 수 있음.', '투명한 유리 파르페 잔에 순서대로: (1) 흑임자 크림 레이어 2큰술, (2) 크런치 1큰술, (3) 에스프레소 젤 또는 에스프레소 1큰술, (4) 휘핑크림 1큰술, 이 과정을 2-3층 반복

## [실습] LLM으로 보고서 개요 생성 후 섹션별 글 작성하기

Structured Output 구조를 활용해, LLM이 주제에 대한 구획을 먼저 구성하고    
해당 구획을 반복문이나 Batch로 각각 입력하여 긴 글을 쓰도록 만들어 보세요.

1. with_structured_output을 통해 주제에 대한 구획 작성하는 체인 `outliner` 만들기
2. 섹션별 글 작성 체인 `writer` 만들기
3. 반복문이나 batch()를 통해 `outliner`의 결과물을 `writer`에 전달하기
4. 최종 결과물 합치기

In [ ]:
# 이 모델을 사용하세요!
gpt_llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x7d40af9147a0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7d40aea64e00>, root_client=<openai.OpenAI object at 0x7d40af09a060>, root_async_client=<openai.AsyncOpenAI object at 0x7d40aecbed20>, model_name='gpt-5-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True, reasoning_effort='low')

In [ ]:
class Sections(BaseModel):
    topic: str = Field(description="글쓰기 주제")
    sections: list[str] = Field(description="주제에 대한 세부 섹션 개요 리스트 (최대 5개 섹션)")

In [ ]:
outliner = gpt_llm.with_structured_output(Sections)
outline = outliner.invoke("""
멀티모달 LLM의 발전 과정에 대한 보고서 개요를 써줘.
각각의 개요는 병렬적 작성이 가능하도록 독립적인 내용을 담아야 하고.
개요만 보고도 내용이 구체적으로 드러나야 해.
""")
outline

Sections(topic='멀티모달 대형언어모델(LLM)의 발전 과정', sections=['역사적 타임라인 및 주요 전환점: (1) 초기 연구 배경 — 멀티모달 처리의 필요성, 멀티채널 신호(음성·이미지·텍스트) 통합 시도; (2) 고전적 방법들 — 특징 기반 결합(fusion)과 분리 처리(pipeline)의 한계; (3) 딥러닝 전환 — CNN/RNN 기반 멀티모달 모델의 등장과 성능 향상; (4) 트랜스포머 도입 — 단일 아키텍처로의 통합 시도(예: ViT+Transformer, CLIP 등); (5) 대형사전학습 시기 — 대규모 멀티모달 코퍼스와 자기지도학습으로 능력 비약; (6) 멀티모달 LLM의 상용화·생태계 확산 — API, 파인튜닝·파라메터 효율화 기법의 확대; (7) 규제·윤리·안전성 이슈가 부각된 전개. 각 단계별 대표 논문·모델(연도, 핵심 기여)과 기술적 한계 및 해결책을 연대기 순으로 정리한다.', '아키텍처 혁신과 설계 패턴: (1) 모달리티별 인코더/브릿지 구조 — 독립 인코더(이미지·오디오·텍스트)와 공통 표현 공간으로의 투영 방식 설명; (2) 조인트 트랜스포머 설계 — 단일 시퀀스로 결합하는 방법, cross-attention/adapter/late fusion의 비교; (3) 멀티스케일·계층적 표현 — 로컬/글로벌 특징을 통합하는 구조적 기법; (4) 파라미터 효율화 — LoRA, Adapter, Prompt tuning 같은 기법의 멀티모달 적용과 장단점; (5) 모달 결손·동적 입력 처리 — 일부 모달 부재 시 동작원리와 학습 기법; (6) 실시간·저지연 요구를 위한 경량화·하드웨어 친화적 설계. 각 항목은 아키텍처 다이어그램 요소, 입력·출력 흐름, 학습 신호(손실 함수) 예시까지 포함하여 독립적으로 기술한다.', '데이터셋·학습전략 및 대규모 사전학습: (1) 멀티모달 코퍼스 구성 — 이미지-캡션, 비디오-자막, 오디오-텍스트, 멀티턴 대화 데이터 등 유형별 수집·정제·라벨링 방법; (2) 자기지도학습 목표들 — 마

In [ ]:
writer_prompt = ChatPromptTemplate([
    ('human','''보고서 주제에 대해, 하나의 섹션에 대한 전문적인 글을 작성하세요.
내용은 최대한 자세하게 쓰고, '입니다' 와 같은 말투로 작성하세요.

---
보고서 전체 주제: {topic}
세부 섹션 주제: {section}
''')
])
writer = writer_prompt | gpt_llm | StrOutputParser()
writer.invoke({'topic':outline.topic, 'section':outline.sections[0]})

'역사적 타임라인 및 주요 전환점\n\n아래 내용은 멀티모달 대형언어모델(LLM)의 발전 과정을 연대기 순으로 정리한 섹션입니다. 각 단계마다 대표 논문·모델(발표연도), 핵심 기여, 당시의 기술적 한계와 이후 해결 시도(또는 제안된 해결책)를 포함합니다.\n\n1) 초기 연구 배경 — 멀티모달 처리의 필요성(1990s–2009)\n- 대표 연구 및 연도\n  - 오디오-비주얼 연동 연구 (예: 오디오-비디오 음성인식, 1990s–2000s)\n  - HMM 기반 멀티채널 음성인식 및 간단한 음성-영상 융합 시도\n  - 텍스트·이미지 결합을 위한 초창기 연구(멀티미디어 정보검색, 2000s)\n- 핵심 기여\n  - 인간의 의사소통은 본질적으로 다중 신호(음성·이미지·텍스트)를 포함한다는 인식이 형성됨.\n  - 서로 다른 센서/모달리티에서 얻는 정보의 상보성(complementarity)을 이용하면 인식 성능을 개선할 수 있다는 실증적 근거 제시.\n  - 멀티채널 특징을 각각 추출해 결합하는 방식(예: 음성의 스펙트럼, 영상의 특징)을 통한 초기 통합 방법들이 등장.\n- 기술적 한계\n  - 모달 간 표현 불일치(heterogeneity): 서로 다른 형태·시간해상도의 신호를 결합하기 어려움.\n  - 데이터 부족: 동기화된 대규모 멀티모달 코퍼스가 부족함.\n  - 기계 학습 모델의 표현력 한계: 당시 모델(예: HMM, GMM, 간단한 신경망)은 복잡한 고차 상호작용을 포착하기 어려움.\n- 해결 시도\n  - 모달리티별 전처리·정규화, 수동적 특징 공학(feature engineering).\n  - 간단한 동기화 및 정렬 알고리즘 개발.\n  - 멀티미디어 데이터셋 수집 노력이 점차 증가.\n\n2) 고전적 방법들 — 특징 기반 결합(fusion)과 분리 처리(pipeline)의 한계(2000s–early 2010s)\n- 대표 연구 및 연도\n  - early fusion vs late fusion 분석(2000s)\n  - 멀티모달 정보검색 

In [ ]:
from langchain_openai import ChatOpenAI
search_llm = ChatOpenAI(model='gpt-4o-search-preview')

writer = writer_prompt.partial(topic=outline.topic) | search_llm | StrOutputParser()
# topic을 미리 채워 매개변수 1개
result = writer.batch(outline.sections)
result

['멀티모달 대형언어모델(LLM)의 발전 과정은 여러 주요 전환점을 거치며 진화해 왔습니다. 각 단계별로 대표적인 논문과 모델, 기술적 한계 및 해결책을 연대기 순으로 정리하겠습니다.\n\n**1. 초기 연구 배경: 멀티모달 처리의 필요성 및 멀티채널 신호 통합 시도**\n\n인간은 시각, 청각, 촉각 등 다양한 감각을 통해 정보를 수집하고 처리합니다. 이러한 멀티모달 정보 처리는 자연스러운 인지 과정의 일부로, 인공지능 시스템에서도 이러한 능력을 모방하려는 시도가 있었습니다. 초기 연구에서는 음성, 이미지, 텍스트 등 여러 채널의 신호를 통합하여 보다 풍부한 표현과 이해를 가능하게 하는 방법들이 탐구되었습니다.\n\n**2. 고전적 방법들: 특징 기반 결합(fusion)과 분리 처리(pipeline)의 한계**\n\n초기 멀티모달 처리 방법은 주로 특징 기반 결합과 분리 처리 방식을 채택했습니다. 특징 기반 결합은 각 모달리티에서 추출된 특징을 결합하여 모델에 입력하는 방식이며, 분리 처리는 각 모달리티를 개별적으로 처리한 후 결과를 통합하는 방식입니다. 그러나 이러한 방법들은 모달리티 간의 상호작용을 충분히 반영하지 못하고, 정보 손실이나 처리 효율성의 문제를 야기했습니다.\n\n**3. 딥러닝 전환: CNN/RNN 기반 멀티모달 모델의 등장과 성능 향상**\n\n딥러닝의 발전으로 컨볼루션 신경망(CNN)과 순차적 신경망(RNN)을 활용한 멀티모달 모델이 등장했습니다. 이러한 모델들은 이미지와 텍스트, 음성 등의 데이터를 효과적으로 처리하고 통합하여 성능을 향상시켰습니다. 예를 들어, 이미지 캡셔닝 모델은 CNN을 통해 이미지 특징을 추출하고, RNN을 통해 텍스트 설명을 생성하는 방식으로 동작했습니다.\n\n**4. 트랜스포머 도입: 단일 아키텍처로의 통합 시도**\n\n트랜스포머 아키텍처의 도입은 멀티모달 처리에 새로운 전환점을 마련했습니다. 트랜스포머는 자기 주의 메커니즘을 통해 장기 의존성을 효과적으로 학습할 수 있어, 다양한 모달리티의 정보를 통합

In [ ]:
draft = '\n\n'.join(result)
with open('result.md', 'w', encoding='utf-8') as f:
    f.write(draft)

print(draft[0:100])

멀티모달 대형언어모델(LLM)의 발전 과정은 여러 주요 전환점을 거치며 진화해 왔습니다. 각 단계별로 대표적인 논문과 모델, 기술적 한계 및 해결책을 연대기 순으로 정리하겠습니다.


<br><br>
## Runnables

LangChain 체인의 기본 구조는 `RunnableSequence` 클래스로 구성됩니다.   

이 때, 시퀀스를 구성한 llm, prompt, chain 각 모듈은 Runnables에 해당합니다.   
Runnables은 자유롭게 체인에 포함되어 결과를 연결할 수 있습니다.



이번에는, 데이터 흐름을 제어하는 특별한 Runnable인   
RunnablePassthrough와 RunnableParallel을 이용해 체인을 구성해 보겠습니다.


<br><br>
### RunnablePassthrough
RunnablePassthrough는 체인의 직전 출력을 그대로 가져옵니다.

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

prompt1 = ChatPromptTemplate(["{director}의 대표 작품은 무엇입니까? 하나의 작품만 선택하고, 해당 작품에 대해 20자 이내로 설명하세요."])
chain1 = (
    prompt1
    | gpt_llm
    | StrOutputParser()
    | {'answer': RunnablePassthrough()})

response = chain1.invoke("스티븐 스필버그")
response

{'answer': '선택 작품: 쉰들러 리스트\n설명 (20자 이내): 홀로코스트 실화 감동'}

<br><br>
### RunnableParallel

RunnableParallel은 서로 다른 체인을 병렬적으로 실행하여 dict 구조로 전달합니다.

In [ ]:
from langchain_core.runnables import RunnableParallel

RunnableParallel()

{
  
}

In [ ]:
prompt1 = ChatPromptTemplate(["색깔을 {n} 알려주세요, 색깔만 출력하세요."])
prompt2 = ChatPromptTemplate(["음식을 {m} 알려주세요, 음식만 출력하세요."])

chain1 = prompt1 | gpt_llm | StrOutputParser()
chain2 = prompt2 | gpt_llm | StrOutputParser()

chain3 = RunnableParallel(color = chain1, food = chain2)
# 딕셔너리에 붙인다

chain3.invoke({'n':5, 'm':3})

{'color': '빨강\n파랑\n초록\n노랑\n보라', 'food': '김치찌개\n비빔밥\n불고기'}

## Assign()

RunnableParallel을 사용하면 중간 체인의 결과를 전달하여, 다음 체인의 결과를 함께 얻을 수 있습니다.   

In [ ]:
prompt1 = ChatPromptTemplate(["잭슨빌은 어느 나라의 도시입니까? 나라 이름만 출력"])
prompt2 = ChatPromptTemplate(
    ["{country}의 대표적인 인물 3명을 나열하세요. 인물의 이름만 출력하세요."]
)

chain1 = prompt1 | gpt_llm | StrOutputParser()
chain2 = prompt2 | gpt_llm | StrOutputParser()

chain3 = RunnableParallel(country = chain1).assign(people = chain2)
#                         {country:'미국}    +      {people:''}
# assign : 직전 체인 결과 (dict)를 뒷 체인에 전달하고 결과를 합침

chain3.invoke({})

{'country': '미국', 'people': '조지 워싱턴\n에이브러햄 링컨\n마틴 루서 킹 주니어'}

<br><br><br><br><br><br><br><br>
chain2에서 새로운 매개변수가 추가되는 경우는 어떻게 해야 할까요?

In [ ]:
prompt1 = ChatPromptTemplate(["{city}는 어느 나라의 도시인가요? 나라 이름만 출력하세요."])
prompt2 = ChatPromptTemplate(["{country}의 유명한 인물은 누가 있나요? {num} 명의 이름을 나열하세요. 사람 이름만 ,로 구분하여 나열하세요."])

chain1 = prompt1 | gpt_llm | StrOutputParser()
# city ---> country

chain2 = (
    RunnablePassthrough.assign(country = chain1)
    # city, num         +      country
    # 입력받은 city, num에 country를 추가하여 전달
    | prompt2
    # country, num을 받아 실행
    | gpt_llm
    | StrOutputParser()
)

print(chain2.invoke({"city": "잭슨빌", "num": "3"}))

Abraham Lincoln, Martin Luther King Jr., Beyoncé Knowles


<br><br>
assign을 여러 개 연결할 수 있습니다.

In [ ]:
chain4 = (prompt2
    | gpt_llm
    | StrOutputParser())

chain3 = RunnablePassthrough.assign(country = chain1).assign(res = chain4)

chain3.invoke({"city": "부에노스 아이레스", "num": "3"})

{'city': '부에노스 아이레스',
 'num': '3',
 'country': '아르헨티나',
 'res': 'Lionel Messi, Eva Perón, Jorge Luis Borges'}

<br><br><br>JsonOutputParser를 쓴다면 아래와 같이 만들 수도 있습니다.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser

prompt1 = ChatPromptTemplate(
    ["영화 배우 한명과 대표작 하나를 출력하세요. json 형식으로 출력하고, 각 항목은 actor, movie로 표시하세요."])
prompt2 = ChatPromptTemplate(["{actor}는 {movie}에서 어떤 역할을 했습니까?"])

chain1 = prompt1 | gpt_llm | JsonOutputParser()
chain2 =(
     chain1 | prompt2 | gpt_llm | StrOutputParser()
)
chain2.invoke({})

'송강호는 영화 《기생충》(감독 봉준호)에서 김기택 역을 맡았습니다.  \n김기택은 김가(家)의 가장으로, 실직 상태이지만 온화하고 상황판단이 빠른 인물입니다. 아들 기우의 소개로 박사장 가문의 운전기사로 들어가면서 가족 전체가 박 가문에 침투하는 이야기가 전개되고, 영화 후반부에는 갈등과 비극적 결말로 이어지는 중심 인물 중 하나입니다.\n\n더 자세한 캐릭터 분석이나 명장면/명대사를 원하시면 말해 주세요.'